# Residential & Commercial demand (historic)

This notebook is prepared to extract historic energy cosumption by residential and commercial sectors. This notebook shall be used along with 'heat_demand_GB_2010.py' file which is based on approach adopted from demandlib: https://github.com/oemof/demandlib/blob/dev/examples/heat_demand_example.py.

Note: current notebook uses only 2010 data of GB hourly temperature fluctuations and share of single-family and multi-family households as of 2021.

In [1]:
import os
import sys
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())
src_path = os.environ.get('PROJECT_SRC')
os.chdir(src_path)
os.getcwd()

'C:\\Users\\tatya\\OneDrive - University of Edinburgh\\01 Dissertation\\01 PyPSA\\PyPSA-GB-H2\\notebooks'

In [2]:
import pandas as pd
import numpy as np

### Extract holidays for selected historic year

In [3]:
from workalendar.europe import UnitedKingdom

cal = UnitedKingdom ()
holidays = dict(cal.holidays(2010))

holidays

{datetime.date(2010, 1, 1): 'New year',
 datetime.date(2010, 4, 2): 'Good Friday',
 datetime.date(2010, 4, 4): 'Easter Sunday',
 datetime.date(2010, 4, 5): 'Easter Monday',
 datetime.date(2010, 5, 3): 'Early May Bank Holiday',
 datetime.date(2010, 5, 31): 'Spring Bank Holiday',
 datetime.date(2010, 8, 30): 'Late Summer Bank Holiday',
 datetime.date(2010, 12, 25): 'Christmas Day',
 datetime.date(2010, 12, 26): 'Boxing Day',
 datetime.date(2010, 12, 27): 'Christmas Shift',
 datetime.date(2010, 12, 28): 'Boxing Day Shift'}

 ### Go to VScode and run 'heat_demand_GB.py' with extracted holidays

Upon results generation from 'heat_demand_GB.py'  to csv. adjust the data to share of residential and commecial hourly consumption. 

### Run below to obtain historic demand fluctuations (in share)

In [4]:
# read csv. file
df = pd.read_csv('..\data\loads\historic-heating\heat_demand_GB_2010.csv')

In [5]:
# sum efh (rural) and mfh (urban) - assumed constant share in 2010 -> 2030, 2050
df['residential'] = df['efh'] + df['mfh']
df = df.rename(columns={'ghd': 'commercial', 'Unnamed: 0': 'name'})
df = df.filter(['name', 'residential', 'commercial'], axis=1)

In [6]:
# add total -> share of total
df.loc['total',:] = df.sum(axis=0, numeric_only=True)
df['residential, share'] = df['residential'].div(df.iloc[8760]['residential']).round(100)
df['commercial, share'] = df['commercial'].div(df.iloc[8760]['commercial']).round(100)
df = df.filter(['name','residential, share','commercial, share'], axis=1)
df = df.drop('total', axis=0)

In [7]:
df

,name,"residential, share","commercial, share"
0,2010-01-01 00:00:00,0.000104,0.000157
1,2010-01-01 01:00:00,0.000109,0.000154
2,2010-01-01 02:00:00,0.000111,0.000175
3,2010-01-01 03:00:00,0.000119,0.000213
4,2010-01-01 04:00:00,0.000148,0.000253
...,...,...,...
8755,2010-12-31 19:00:00,0.000194,0.000169
8756,2010-12-31 20:00:00,0.000189,0.000160
8757,2010-12-31 21:00:00,0.000171,0.000141
8758,2010-12-31 22:00:00,0.000142,0.000126


In [8]:
# write 'heat_demand_GB_2010-share'
df.to_csv('..\data\loads\historic-heating\heating-historic-output.csv', index=False)